In [1]:
%load_ext autoreload
%autoreload 2
%gui qt
%matplotlib qt5

import sys
import os

import numpy as np
import matplotlib.pyplot as plt

### Load Metadata module

In [2]:
from oyLabImaging import Metadata

### Define path

In [3]:
fpath = "/bigstore/TestSpin/08132021_TrackingCells_mRubyLoss/_1/"
#fpath = "/bigstore/Microscopy Core/Jen/A549Inf_Drugs.nd2"
#fpath="/bigstore/Microscopy Core/Jen/DIC_timelapseDeathComparison_08032021.pickle"
#fpath ='/bigstore/TestSpin/08132021_TrackingCells_mRubyLoss/_1/metadata.pickle'
#fpath = '/bigstore/Microscopy Core/Jen/3T3_mRubyloss_HSV_20210818/'
#fpath = '/bigstore/Microscopy Core/Jen'
#fpath = '/bigstore/Microscopy Core/Jen/3T3_mRubyloss_HSV_20210818/MRubyVirusDeath_08172021.nd2'
MD = Metadata(fpath)

loaded MM metadata from/bigstore/TestSpin/08132021_TrackingCells_mRubyLoss/_1/metadata.pickle


### You can read images by attributes

In [29]:
stk = MD.stkread(Channel = MD.channels[0], Position=MD.posnames[2],register=True)

opening file img_channel000_position001_time000000059_z000.tif
Loaded Pos1 group of images.


In [25]:
MD.CalculateDriftCorrection(Position='Pos2')

opening file img_channel000_position002_time000000059_z000.tif
Loaded Pos2 group of images.

calculating drift correction for position Pos2
calculated drift correction for position Pos2


### use napari to see images

In [5]:
import napari
viewer = napari.Viewer()

2021-09-08 14:50:20,695 [INFO] WRITING LOG OUTPUT TO /home/alo649/.cellpose/run.log


In [6]:
viewer.add_image(stk)

<Image layer 'stk' at 0x7fe288d70d30>

### If you want to segment cells, you can use a function called test_segmentation_params to see how the algorythm performs with different parameters. Any parameter value that's changed from the default should be manually added to the call for segmentation of the full data below.

In [29]:
from oyLabImaging.Processing.improcutils import segmentation
print(segmentation.segmentation_types()) 
img = MD.stkread(Position=MD.posnames[1],Channel=MD.channels[0], frame=10)
segmentation.test_segmentation_params(img=img, segment_type='cellpose_nuclei', cellprob_threshold=0.0)#cellpose_nuclei

['watershed', 'cellpose_nuclei', 'cellpose_cyto', 'cellpose_nuc_cyto']
opening file img_channel000_position000_time000000010_z000.tif
Loaded Pos0 group of images.

using _segment_nuclei_cellpose


calculating with new parameters
Done!
calculating with new parameters
Done!
calculating with new parameters
Done!
calculating with new parameters
Done!


### make empty results object

In [4]:
from oyLabImaging.Processing import results
R = results(MD=MD)


loaded results from pickle file


In [5]:
R()

Results object for path to experiment in path: 
 /bigstore/TestSpin/08132021_TrackingCells_mRubyLoss/_1/

Available channels are : DeepBlue, Green, Red.

Positions already segmented are : Pos0, Pos1

Available positions : Pos2, Pos0, Pos1.

Available frames : 60.


In [11]:
R.setPosLbls(MD=MD,Position='Pos1',NucChannel='DeepBlue',segment_type='cellpose_nuclei', threads=6, diameter=70)


Processing position Pos1


100%|███████████████████████████████████████████| 60/60 [00:33<00:00,  1.81it/s]



Finished loading and segmenting position Pos1
saved results


In [9]:
R.PosLbls['Pos1'].framelabels[0]._seg_params

{'diameter': 70, 'scale': 0.5, 'GPU': True}

In [7]:
R.calculate_tracks(pos='Pos1', NucChannel='DeepBlue', search_radius=25, maxStep=30)

linking frame 58
Finished connecting tracks
saved results


In [32]:
R.show_points('Pos1')

2021-09-08 16:16:27,751 [INFO] WRITING LOG OUTPUT TO /home/alo649/.cellpose/run.log


In [33]:
R.show_tracks('Pos1')

In [ ]:
t0 = R.tracks('Pos0')

In [ ]:
i=20
plt.plot(t0(i).T, t0(i).mean('DeepBlue'))
plt.show()